In [ ]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

In [1]:
import pandas as pd

headers = ["id", "date", "name", "text", "type", "rep", "fav", "stcount", "fol", "frien", "listcount"]
neg_df = pd.read_csv("negative.csv", sep=";", names=headers, index_col=False)
pos_df = pd.read_csv("positive.csv", sep=";", names=headers, index_col=False)

In [2]:
neg_df.head()

,id,date,name,text,type,rep,fav,stcount,fol,frien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34


In [3]:
pos_df.head()

,id,date,name,text,type,rep,fav,stcount,fol,frien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23


In [4]:
import re

def rm_punct(text):
    res = re.sub(r"[^\w\s]", " ", text.lower())
    res = re.sub(r"\s+", " ", res)
    return res.strip()

In [5]:
neg_df["text_rm_punct"] = neg_df["text"].apply(rm_punct)
pos_df["text_rm_punct"] = pos_df["text"].apply(rm_punct)

In [6]:
neg_texts = neg_df.text_rm_punct.values
pos_texts = pos_df.text_rm_punct.values

In [7]:
pos_texts

array(['first_timee хоть я и школота но поверь у нас то же самое d общество профилирующий предмет типа',
       'да все таки он немного похож на него но мой мальчик все равно лучше d',
       'rt katiacheh ну ты идиотка я испугалась за тебя', ...,
       'что происходит со мной когда в эфире proactivefm звучит моя любимая песня dctalk music http t co 65kgffd5oo',
       'любимая я подарю тебе эту звезду имя какой звезды переводится подмышка',
       'ma_che_rie посмотри непытайтесьпокинутьомск сегодня в вавилоне в 18 20 я там тоже есть'],
      dtype=object)

In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, TimeDistributed, InputLayer
from tensorflow.keras.layers import Embedding, Conv1D, Input, concatenate, SpatialDropout1D, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

In [9]:
import numpy as np

texts = np.concatenate([pos_texts, neg_texts])
target = np.concatenate([pos_df["type"].values, neg_df["type"].values])

X_train, X_test, y_train, y_test = train_test_split(texts, target, test_size=.2, random_state=1337)

In [10]:
X_test

array(['lentaruofficial ну тут всё просто либо либо',
       'мама когда же ты уже научишься себя беречь',
       'rt yprtcrazy_mofo baisa04 ты мой 469 читатель добра тебе http t co lkm5nwykvm',
       ...,
       'q ты самая красивая девушка из 10 классов хотя и во всей школе a http t co rkdeczokvj',
       'kks35he2013 охренеть а я только анг и историю и то додел надо',
       'satstranger поживём увидим чё они там предложат но мне было бы интересно полапать такой девайс'],
      dtype=object)

# Архитектура 1

модель берет слова, пропускает их через Embedding слой. По эмбеддингам проходит biLSTM, на выходе линейный слой и выходной слой.

Embedding слой обучается внутри модели -- 1 балл

подгружаются обученные эмбеддинги для русского языка ** -- 1 балл

fasttext эмбеддинги обучаются на всем корпусе с нуля *** -- 2 балла

In [11]:
vocab = Counter()
for sent in X_train:
    sent = [word.lower() for word in sent.split()]
    vocab.update(sent)

In [12]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}

word2id = {word: i + 2 for i, word in enumerate(filtered_vocab)}
word2id['pad'] = 0
word2id['unk'] = 1  

id2word = {i: word for word, i in word2id.items()}

In [13]:
def data2ints(data, smth2id):
    int_data = []
    for seq in data:
        int_seq = []
        for i in seq:
            int_seq.append(smth2id.get(i.lower(), 1))
  
        int_data.append(int_seq)
    return int_data

In [14]:
X_train_ids, X_test_ids = data2ints(X_train, word2id), data2ints(X_test, word2id)

In [15]:
sent_max_len = max(len(x) for x in X_train_ids)

sent_max_len

173

In [16]:
X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')

In [17]:
y_test_replaced = [elem if elem == 1 else 0 for elem in y_test]
y_train_replaced = [elem if elem == 1 else 0 for elem in y_train]

y_train_cat = to_categorical(y_train_replaced, num_classes=2)
y_test_cat = to_categorical(y_test_replaced, num_classes=2)

In [18]:
X_train.shape, y_train_cat.shape, X_test.shape, y_test_cat.shape

((181467, 173), (181467, 2), (45367, 173), (45367, 2))

In [19]:
import tensorflow as tf

metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()]

In [ ]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=20, input_length=sent_max_len, mask_zero=True)(word_in)
lstm = Bidirectional(LSTM(units=128))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 173)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 173, 20)           522560    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               152576    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 675,650
Trainable params: 675,650
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

Epoch 1/2
1418/1418 [==============================] - 1644s 1s/step - loss: 0.6263 - precision: 0.6246 - recall: 0.6246 - accuracy: 1.0677e-07 - val_loss: 0.5911 - val_precision: 0.6603 - val_recall: 0.6603 - val_accuracy: 1.8736e-04
Epoch 2/2
1418/1418 [==============================] - 1702s 1s/step - loss: 0.5702 - precision: 0.6795 - recall: 0.6795 - accuracy: 2.0750e-04 - val_loss: 0.5174 - val_precision: 0.7217 - val_recall: 0.7217 - val_accuracy: 8.9272e-04


С accuracy проблема: модель предсказывает числа типа 9.7562476e-1, а сравнивает с 0 и 1. Давайте округлим и пересчитаем

In [ ]:
pred = model.predict(X_test[:1000])  # полностью предсказывает долго

rounded_pred = []
for pair in pred:
  rounded_pred.append([round(pair[0]), round(pair[1])])

In [34]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [ ]:
prec = precision_score(rounded_pred, y_test_cat[:1000], average="macro")
rec = recall_score(rounded_pred, y_test_cat[:1000], average="macro")
acc = accuracy_score(rounded_pred, y_test_cat[:1000])

print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")

Precision: 0.7363157810524968
Recall: 0.742206333327867
Accuracy: 0.736


Я сохраню здесь вывод ячейки выше:

Precision: 0.7363157810524968
Recall: 0.742206333327867
Accuracy: 0.736

# Архитектура 2

In [ ]:
# изменим размерность эмбеддингов и лстм, добавим дропаут

word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=50, input_length=sent_max_len, mask_zero=True)(word_in)
lstm = Bidirectional(LSTM(units=256, recurrent_dropout=0.15))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 173)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 173, 50)           1306400   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               628736    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
Total params: 1,936,162
Trainable params: 1,936,162
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

Epoch 1/2
1418/1418 [==============================] - 7134s 5s/step - loss: 0.6273 - precision: 0.6789 - recall: 0.6789 - accuracy: 4.1561e-04 - val_loss: 0.5550 - val_precision: 0.6974 - val_recall: 0.6974 - val_accuracy: 0.0000e+00
Epoch 2/2
1418/1418 [==============================] - 7072s 5s/step - loss: 0.5326 - precision: 0.7188 - recall: 0.7188 - accuracy: 9.0896e-05 - val_loss: 0.5053 - val_precision: 0.7380 - val_recall: 0.7380 - val_accuracy: 1.6532e-04


Очень медленно обучается (ну, в целом, ожидаемо), оставила модель в колабе на ночь

In [ ]:
pred = model.predict(X_test[:1000])

rounded_pred = []
for pair in pred:
  rounded_pred.append([round(pair[0]), round(pair[1])])


prec = precision_score(rounded_pred, y_test_cat[:1000], average="macro")
rec = recall_score(rounded_pred, y_test_cat[:1000], average="macro")
acc = accuracy_score(rounded_pred, y_test_cat[:1000])

print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")

Precision: 0.7446639146226339
Recall: 0.7519410977242302
Accuracy: 0.745


Precision: 0.7446639146226339
Recall: 0.7519410977242302
Accuracy: 0.745

# Архитектура 3

Сначала я прерывала обучение архитектуры 2, т.к. оно было очень медленным, поэтому эти результаты получила раньше, чем те, что выше. Так что формально это 2, а 2 — это 3, но не думаю, что нумерация на что-то влияет

In [ ]:
# вернем размерности, оставим дропаут

word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=20, input_length=sent_max_len, mask_zero=True)(word_in)
lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.15))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

In [ ]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

Epoch 1/2
1418/1418 [==============================] - 2316s 2s/step - loss: 0.6232 - precision: 0.6386 - recall: 0.6386 - accuracy: 1.4995e-04 - val_loss: 0.5279 - val_precision: 0.7196 - val_recall: 0.7196 - val_accuracy: 5.5106e-05
Epoch 2/2
1418/1418 [==============================] - 2315s 2s/step - loss: 0.5319 - precision: 0.7179 - recall: 0.7179 - accuracy: 2.6501e-04 - val_loss: 0.4893 - val_precision: 0.7496 - val_recall: 0.7496 - val_accuracy: 0.0010


In [ ]:
pred = model.predict(X_test[:1000])

rounded_pred = []
for pair in pred:
  rounded_pred.append([round(pair[0]), round(pair[1])])


prec = precision_score(rounded_pred, y_test_cat[:1000], average="macro")
rec = recall_score(rounded_pred, y_test_cat[:1000], average="macro")
acc = accuracy_score(rounded_pred, y_test_cat[:1000])

print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")

Precision: 0.765988255812093
Recall: 0.7660010240655402
Accuracy: 0.766


Precision: 0.765988255812093
Recall: 0.7660010240655402
Accuracy: 0.766

У архитектуры 1 (базовой) все метрики ~0.74, у архитектуры 2 — 0.74-0.75, у архитектуры 3 — 0.76-0.77. Вывод: с размером эмбеддингов 20, 128 нейронами в LSTM + дропаутом 0.15 получается лучше всего.

# Готовые эмбеддинги

In [20]:
from gensim.models import KeyedVectors

araneum_fasttext = KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')

In [26]:
weights = np.array([araneum_fasttext[id2word[id_]] for id_ in id2word])

In [27]:
weights.shape

(26128, 300)

In [29]:
# возьмем за основу архитектуру 3


word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.15))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

In [30]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 173)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 173, 300)          7838400   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 8,278,210
Trainable params: 439,810
Non-trainable params: 7,838,400
_________________________________________________________________


In [31]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=1, verbose=1)

1418/1418 [==============================] - 465s 328ms/step - loss: 0.6149 - precision: 0.6387 - recall: 0.6387 - accuracy: 0.0000e+00 - val_loss: 0.5229 - val_precision: 0.7157 - val_recall: 0.7157 - val_accuracy: 0.0000e+00


In [32]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=1, verbose=1)

1418/1418 [==============================] - 465s 328ms/step - loss: 0.4968 - precision: 0.7415 - recall: 0.7415 - accuracy: 0.0000e+00 - val_loss: 0.4736 - val_precision: 0.7599 - val_recall: 0.7599 - val_accuracy: 0.0000e+00


In [35]:
pred = model.predict(X_test[:1000])

rounded_pred = []
for pair in pred:
  rounded_pred.append([round(pair[0]), round(pair[1])])


prec = precision_score(rounded_pred, y_test_cat[:1000], average="macro")
rec = recall_score(rounded_pred, y_test_cat[:1000], average="macro")
acc = accuracy_score(rounded_pred, y_test_cat[:1000])

print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")

Precision: 0.7819645114321829
Recall: 0.7820728291316527
Accuracy: 0.782


Получилось еще лучше!

# Эмбеддинги fasttext с нуля

In [41]:
corpus = np.concatenate([pos_texts, neg_texts])

In [48]:
import gensim

ft = gensim.models.FastText(corpus, size=300, iter=5)

In [49]:
ft_weights = np.array([ft[id2word[id_]] for id_ in id2word])

<ipython-input-49-c068eb0099b7>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  ft_weights = np.array([ft[id2word[id_]] for id_ in id2word])


In [50]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[ft_weights])(word_in)
lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.15))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

In [51]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 173)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 173, 300)          7838400   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 8,278,210
Trainable params: 439,810
Non-trainable params: 7,838,400
_________________________________________________________________


In [52]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

Epoch 1/2
1418/1418 [==============================] - 465s 328ms/step - loss: 0.6986 - precision: 0.5544 - recall: 0.5544 - accuracy: 0.0000e+00 - val_loss: 0.7052 - val_precision: 0.4914 - val_recall: 0.4914 - val_accuracy: 0.0000e+00
Epoch 2/2
1418/1418 [==============================] - 466s 328ms/step - loss: 0.6972 - precision: 0.5022 - recall: 0.5022 - accuracy: 0.0000e+00 - val_loss: 0.6940 - val_precision: 0.5086 - val_recall: 0.5086 - val_accuracy: 0.0000e+00


In [53]:
pred = model.predict(X_test[:1000])

rounded_pred = []
for pair in pred:
  rounded_pred.append([round(pair[0]), round(pair[1])])


prec = precision_score(rounded_pred, y_test_cat[:1000], average="macro")
rec = recall_score(rounded_pred, y_test_cat[:1000], average="macro")
acc = accuracy_score(rounded_pred, y_test_cat[:1000])

print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")

Precision: 0.5
Recall: 0.251
Accuracy: 0.502


/home/grafd/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


штош

In [54]:
ft = gensim.models.FastText(corpus, size=300, iter=20)

In [55]:
ft_weights = np.array([ft[id2word[id_]] for id_ in id2word])

word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[ft_weights])(word_in)
lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.15))(emb_word)
out = Dense(2, activation="softmax")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)

model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

<ipython-input-55-f5367c4f4ae0>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  ft_weights = np.array([ft[id2word[id_]] for id_ in id2word])


Epoch 1/2
1418/1418 [==============================] - 466s 328ms/step - loss: 0.7081 - precision: 0.5018 - recall: 0.5018 - accuracy: 0.0000e+00 - val_loss: 0.6932 - val_precision: 0.4914 - val_recall: 0.4914 - val_accuracy: 0.0000e+00
Epoch 2/2
1418/1418 [==============================] - 466s 328ms/step - loss: 0.7066 - precision: 0.5012 - recall: 0.5012 - accuracy: 0.0000e+00 - val_loss: 0.7327 - val_precision: 0.5086 - val_recall: 0.5086 - val_accuracy: 0.0000e+00


Увеличение числа эпох фасттекста не помогло :(

# Два входа

In [56]:
chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
print(chars)

{'т', 'n', 'в', 'e', 'и', 't', 'а', 'г', 'ї', 'q', 'u', 'k', 'щ', 'ë', 'g', 'x', 'і', 'ё', '6', 'м', 'ы', 's', 'ғ', 'd', 'р', 'ю', '0', 'є', 'х', 'j', 'w', 'л', 'o', 'ъ', '3', 'к', '7', 'ь', 'b', 'я', 'z', 'p', 'v', 'у', '9', 'й', 'a', '1', 'э', 'қ', 'е', 'п', 'f', 'c', 'с', 'ө', 'б', 'i', 'h', 'ч', 'ʘ', 'ф', 'l', '5', 'ц', 'з', 'ш', 'r', 'ў', '_', 'ж', 'д', 'ү', 'y', 'm', '2', 'о', 'н', '4', 'ң', '8'}


In [57]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i: char for char, i in char2id.items()}

In [59]:
char_max_len = max(len(x) for x in filtered_vocab)
print("максимальная длина слова:", char_max_len)

максимальная длина слова: 33


In [66]:
X_train, X_test, y_train, y_test = train_test_split(texts, target, test_size=.2, random_state=1337)

X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')

In [67]:
def make_X_char(sentences):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(sent_max_len):
          word_seq = []
          for j in range(char_max_len):
              try:
                  word_seq.append(char2id[sentence[i][j].lower()])
              except:
                  word_seq.append(char2id["pad"])
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
  return np.array(X_char)


# срез, потому что полностью очень долго
X_char_train, X_char_test = make_X_char(X_train[:1000]), make_X_char(X_test[:500])

In [75]:
# возьмем готовые эмбеддинги 
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)

char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128,
                               recurrent_dropout=0.15)
                         )(x)
out = Dense(2, activation="softmax")(main_lstm)

model = Model(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=metrics)

In [76]:
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 173, 33)]    0                                            
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, 173, 33, 10)  830         input_17[0][0]                   
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 173)]        0                                            
__________________________________________________________________________________________________
time_distributed_10 (TimeDistri (None, 173, 31, 12)  372         time_distributed_9[0][0]         
_______________________________________________________________________________________

In [77]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), batch_size=128, epochs=2, verbose=1)

Epoch 1/2


ValueError: in user code:

    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/grafd/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:789 ru

In [87]:
!pip install graphviz pydot

You should consider upgrading via the '/home/grafd/venv/bin/python -m pip install --upgrade pip' command.


In [86]:
from keras.utils.vis_utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


Я ничего не понимаю..... Я пыталась понять, но у меня не получилось.... Очень грустно